## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [ ]:
import warnings

from cp_omero_toolbox import run_dataset_cp_pipeline, run_project_cp_pipeline
from omero.gateway import BlitzGateway

warnings.filterwarnings('ignore')

import ezomero

# Make CellProfiler run without a GUI
import cellprofiler_core.preferences as cp_preferences
cp_preferences.set_headless()

from cp_omero_toolbox import *

import tempfile

In [ ]:
# Tell CellProfiler to get input from and save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
input_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)
cp_preferences.set_default_image_directory(input_dir.name)

Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [ ]:
# Creating a connection object
host = input("Host: ")
port = int(input("Port (4064): ") or 4064)
group = input("Group: ")

conn = BlitzGateway(host=host, port=port, group=group, secure=True)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that we can tell our new connection to say "Hi, I'm still here" every 60 seconds
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

Time to grasp a Dataset from OMERO and download a CellProfiler pipeline that is attached to it. Go to the browser, select a dataset and copy the ID.
First define if you want to analyze a dataset or a project

In [ ]:
object_type = input("Object type (dataset/project): ")
if object_type == "dataset":
    dataset_id = int(input("Dataset id: "))
    project_id = None
elif object_type == "project":
    dataset_id = None
    project_id = int(input("Project id: "))
else:
    raise ValueError("Object type must be either 'dataset' or 'project'")

We can now run a function that will do all the necessary steps
- Download the pipeline that is attached to the specified dataset.
- Modify the pipeline to run headless (without a GUI)
- Run the analysis pipeline on every image in the dataset
- Upload the results to OMERO
  - Tables (measurements)
  - ROIs (masks and points)
- Gather the results and return them as a dictionary in case you want to do something with them

In [ ]:
if object_type == "dataset":
    measurements_dfs = run_dataset_cp_pipeline(
        conn=conn,
        dataset_id=dataset_id,
        objects_to_image_table="Nuclei",
        objects_to_mask=["Nuclei"],
        link_to_project=False,
        link_to_dataset=True,
        link_to_image=False,
        populate_key_value_pairs=True,
        objects_to_point=["spots"],
        output_dir=output_dir,
        input_dir=input_dir
    )
elif object_type == "project":
    measurements_dfs = run_project_cp_pipeline(
        conn=conn,
        project_id=project_id,
        objects_to_image_table="Nuclei",
        objects_to_mask=["Nuclei"],
        link_to_project=True,
        link_to_dataset=False,
        link_to_image=False,
        populate_key_value_pairs=True,
        objects_to_point=["spots"],
        output_dir=output_dir,
        input_dir=input_dir
    )

After the analysis is done. We can have a look at the results in OMERO.

And we can also have a look at the results in the notebook

In [ ]:
cols = measurements_dfs["Image"].columns
vals = [measurements_dfs["Image"][c].values.tolist() for c in cols]

print(len(vals))
print(vals)



And we should clean up after ourselves

In [6]:
# remove the output and input directories
output_dir.cleanup()
input_dir.cleanup()

# and close the connection to the OMERO server
conn.close()